# Max Flow: Ford-Fulkerson 

In [2]:
immutable Arc
    from::Int
    to::Int
    capacity::Int
    flow::Int
end

In [6]:
adjL=[[Arc(1,2,15,0),Arc(1,3,10,0)],
    [Arc(2,4,8,0)],
    [Arc(3,2,4,0),Arc(3,5,8,0)],
    [Arc(4,6,7,0),Arc(4,5,9,0)],
    [Arc(5,6,20,0)],
    Arc[]
]

6-element Array{Array{Arc,1},1}:
 Arc[Arc(1,2,15,0),Arc(1,3,10,0)]
 Arc[Arc(2,4,8,0)]               
 Arc[Arc(3,2,4,0),Arc(3,5,8,0)]  
 Arc[Arc(4,6,7,0),Arc(4,5,9,0)]  
 Arc[Arc(5,6,20,0)]              
 Arc[]                           

In [7]:
function createResidualNetwork(adjL)
    residN=deepcopy(adjL)
    for i in eachindex(adjL)
        for arc in adjL[i]
            from=arc.to
            to=arc.from
            newarc=Arc(from,to,arc.flow,0)
            push!(residN[from],newarc)
        end
    end
    return residN
end

createResidualNetwork (generic function with 1 method)

In [8]:
createResidualNetwork(adjL)

6-element Array{Array{Arc,1},1}:
 Arc[Arc(1,2,15,0),Arc(1,3,10,0)]            
 Arc[Arc(2,4,8,0),Arc(2,1,0,0),Arc(2,3,0,0)] 
 Arc[Arc(3,2,4,0),Arc(3,5,8,0),Arc(3,1,0,0)] 
 Arc[Arc(4,6,7,0),Arc(4,5,9,0),Arc(4,2,0,0)] 
 Arc[Arc(5,6,20,0),Arc(5,3,0,0),Arc(5,4,0,0)]
 Arc[Arc(6,4,0,0),Arc(6,5,0,0)]              

In [9]:
function find_addmissable!(L,marked_list,currentArc,tail)
    for i=currentArc[tail]:length(L[tail]) # Bad way of doing it woud be: for head in L[tail][currentArc[tail]:end]
        arc=L[tail][i]
        head=arc.to # CHANGED for new data structure 
        if arc.capacity>0 && !marked_list[head]
            currentArc[tail]=i+1 # We modify the currentArc
            return head
        end
    end
    return -1 # Special value to indicate no addmissable arc 
end

function findpath(s,t,aL)
    # Initialization
    n=length(aL) #number of nodes 
    pred=zeros(Int,n)
    marked=falses(n) #CHANGED 
    marked[s]=true # ADDED 
    currentArc=ones(Int,n) # ADDED 
    List=Int[s]
    path=Int[]
    # Now I can stard doing the actual work 
    while !isempty(List)
        i=List[end]#This is the current node
        j=find_addmissable!(aL,marked,currentArc,i) #Changed 
        if j>0 # We found an addmissable arc 
            marked[j]=true
            pred[j]=i
            push!(List,j)
        else
            pop!(List)
        end
        if j==t
            # put some unshit to buid path in a nice loop!
        end
    end
    path
end

graph_search (generic function with 2 methods)

In [11]:
graph_search(1,adjL)

6-element Array{Int64,1}:
 0
 1
 1
 2
 4
 4

In [13]:
unshift!([1,2],3)

3-element Array{Int64,1}:
 3
 1
 2